In [1]:
import sys
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")

from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *

In [2]:
raw_data = pd.read_csv("../raw_data/all_game_data_v1_corrected2.csv", low_memory=False)

In [3]:
reference_data = pipeline_genre_ohe_only(raw_data)

/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [4]:
reference_data['description'] = reference_data['description'].astype(str)

In [5]:
extended_stop_words = ['able','access','across','also','always','another','away','back','become','best','better','big','box','bring','certain','clear','close','come','console','content','could','course','digital','dont','one','two','three','four','five','six','seven','eight','nine','ten','hundred','thousand','either','enjoy','enough','even','exclusive','extra','feature','franchise','full','fully','fun','game','gameplay','genre','get','give','go','good','great','great','greatest','happen','however','huge','ii','improve','include','increase','inside','interactive','introduce','instead','involve','large','last','later','launch','lead','let','level','like','little','look','main','meet','might','mix','modern','mode','must','name','new','next','need','number','nintendo','official','offer','object','option','order','original','originally','others','part','pc','perfect','platform','playable','player','playstation','plus','possible','port','prepare','previous','progress','project','put','reach','ready','remain','return','screen','scroll','second','first','third','see','sega','sequel','series','set','several','show','side','similar','since','small','something','sound','special','start','stat','state','stay','still','studio','super','take','switch','tell','test','th','though','throughout','title','together','try','ultimate','unique','update','upon','us','use','version','via','wait','want','wii','within','without','would','xbox','youll','youre','youve']

In [6]:
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
    stop_words = stopwords.words('english') ## define stopwords
    stop_words.extend(extended_stop_words)
    
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    
    noun_lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "n") 
        for word in tokenized_sentence_cleaned
    ]
    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in noun_lemmatized
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [7]:
reference_data['clean_text'] = reference_data['description'].apply(preprocessing)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.02)

vectorized_descriptions = vectorizer.fit_transform(reference_data['clean_text'])
vectorized_descriptions = pd.DataFrame(
    vectorized_descriptions.toarray(), 
    columns = vectorizer.get_feature_names_out()
)

vectorized_descriptions

,ability,action,add,advance,adventure,alien,allow,along,ancient,arcade,...,visual,war,way,weapon,well,win,work,world,year,young
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.257153,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.252426,0.0,0.0,0.164191,0.000000,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.573865,0.0
36816,0.266112,0.0,0.000000,0.0,0.0,0.0,0.0,0.282949,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
36817,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.150845,0.000000,0.0
36818,0.000000,0.0,0.612294,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.433358,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [14]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate the LDA 

lda_model = LatentDirichletAllocation(n_components=10, max_iter = 50, learning_method='online',)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_descriptions)

LatentDirichletAllocation(learning_method='online', max_iter=50)

In [15]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [16]:
print_topics(lda_model, vectorizer)

Topic 0:
[('publish', 586.8672639242731), ('online', 417.33575967309037), ('include', 392.943740120872), ('edition', 344.5097015469765), ('match', 275.64797491265847), ('well', 273.6873476683612), ('multiplayer', 273.0259743475524), ('opponent', 268.50359035087246), ('fly', 263.87930219606454), ('game', 258.39794011884976)]
Topic 1:
[('shoot', 525.227754808066), ('enemy', 469.6983539199594), ('level', 398.0415108181091), ('move', 332.2980396601159), ('point', 330.516253653566), ('weapon', 319.11928837345556), ('mission', 303.3691926558599), ('jump', 264.8824491892956), ('pack', 263.4275771828037), ('collect', 258.11577690690444)]
Topic 2:
[('release', 1153.7350915429395), ('game', 625.7014258299098), ('video', 615.2217341261158), ('develop', 601.011700455477), ('arcade', 512.7437634442575), ('japan', 473.2296032461051), ('base', 410.7288892761831), ('horror', 399.67946500955236), ('music', 340.7252560125141), ('party', 285.0336673991976)]
Topic 3:
[('race', 780.0260593420354), ('shoote

In [39]:
reference_data[['clean_text']]

,clean_text
0,multiple side pick poison choose sweet siniste...
1,anger foot lightningfast hardbass blast kick d...
2,life feudal mmo openworld survival take player...
3,play history behind scene jordan mechners land...
4,mia paranormal detective search miss father pe...
...,...
36815,year ad galactic federal government agree full...
36816,create titchuk release flash portal november a...
36817,sea battle board everybody love childhood opti...
36818,control mind insert subliminal message video c...


In [45]:
vectorized_example = reference_data['clean_text'][36819]

In [46]:
vectorized_example

'break hit meow play karate cat world paw softest'

In [47]:
ve2 = vectorizer.transform(pd.Series(vectorized_example))
ve2

<1x220 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [48]:
vec_array = lda_model.transform(ve2)

/home/kieran/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(


In [49]:
vec_array[0].tolist()

[0.03732887394866238,
 0.03732748202730333,
 0.03732717647946928,
 0.0373296981405651,
 0.03732938230142989,
 0.03733684307893798,
 0.03733045641347459,
 0.664033766236152,
 0.037324263045383506,
 0.03733205832862182]

In [50]:
vec_array[0].tolist().index(max(vec_array[0].tolist()))

7

In [53]:
def nlp_topic(x):
    vectorized_x = vectorizer.transform(pd.Series(x))
    vec_x_array = lda_model.transform(vectorized_x)
    vec_list = vec_x_array[0].tolist()
    return vec_list.index(max(vec_list))
    

In [54]:
reference_data['topic'] = reference_data['clean_text'].apply(nlp_topic)

/home/kieran/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(


In [55]:
reference_data[['topic']].value_counts()

topic
9        7220
3        5380
5        4359
7        3953
1        3843
2        3737
0        3673
6        2228
8        1503
4         924
dtype: int64

In [56]:
reference_data[['topic']]

,topic
0,6
1,7
2,5
3,2
4,4
...,...
36815,4
36816,7
36817,3
36818,6


In [62]:
pd.get_dummies(reference_data['topic'])

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
36815,0,0,0,0,1,0,0,0,0,0
36816,0,0,0,0,0,0,0,1,0,0
36817,0,0,0,1,0,0,0,0,0,0
36818,0,0,0,0,0,0,1,0,0,0


In [63]:
topics_for_36k_games = pd.get_dummies(reference_data['topic'])

In [67]:
topics_for_36k_games.to_csv("../raw_data/topics.csv", index=False)